In [6]:
import os
import requests
import pandas as pd
import random
import math
import geopandas as gpd
from shapely.geometry import Point

# Parameters
csv_file = "seamounts.csv"  # Path to the .csv file
output_folder = "no_seamounts_galore"  # Folder to save downloaded images
tile_pixels = 800  # Width and height of the image in pixels
tile_width_deg = 1.0  # Fixed width in degrees
exclusion_radius_deg = 2.0  # Exclusion zone around each point (in degrees)
api_base_url = "https://www.gmrt.org/services/ImageServer"
max_attempts = 10  # Max number of attempts to find valid points

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Load CSV file with the points to avoid
try:
    df = pd.read_csv(csv_file)
    print(f"Loaded {len(df)} records from {csv_file}.")
except Exception as e:
    print(f"Error loading .csv file: {e}")
    exit()

# Create a list of points to avoid (exclusion zones)
exclude_points = [(row["LAT"], row["LONG"]) for index, row in df.iterrows()]
print(f"Exclusion zones defined for {len(exclude_points)} points.")

# Load the Natural Earth ocean shapefile
# You'll need to download this shapefile from Natural Earth
ocean_shapefile = "ne_10m_ocean.shp"  # Change this to the path where you saved the shapefile
oceans = gpd.read_file(ocean_shapefile)

# Function to check if a location is in the ocean using geospatial data
def is_ocean(lat, lon):
    point = Point(lon, lat)  # Create a Point object with longitude and latitude
    return oceans.contains(point).any()  # Check if the point is in the ocean

# Function to generate random points outside exclusion zones
def generate_random_point():
    # Generate random latitudes and longitudes
    lat = random.uniform(-90, 90)
    lon = random.uniform(-180, 180)
    
    # Check if the point is within any exclusion zone
    for excl_lat, excl_lon in exclude_points:
        distance = math.sqrt((lat - excl_lat)**2 + (lon - excl_lon)**2)
        if distance < exclusion_radius_deg:
            return generate_random_point()  # Recursively try again if within exclusion zone
    
    return lat, lon

# Function to download the image tile
def download_tile(center_lat, center_lon, file_name):
    # Adjust height based on Mercator projection
    adjusted_height_deg = tile_width_deg * math.cos(math.radians(center_lat))
    
    # Calculate tile bounds
    minlatitude = center_lat - (adjusted_height_deg / 2)
    maxlatitude = center_lat + (adjusted_height_deg / 2)
    minlongitude = center_lon - (tile_width_deg / 2)
    maxlongitude = center_lon + (tile_width_deg / 2)

    # Prepare API parameters
    params = {
        "minlatitude": minlatitude,
        "maxlatitude": maxlatitude,
        "minlongitude": minlongitude,
        "maxlongitude": maxlongitude,
        "width": tile_pixels,
        "height": tile_pixels,  # Ensure the image is square
        "mask": "false",
        "download": "true",
    }

    # Construct file name
    output_file = os.path.join(output_folder, f"{file_name}.png")

    # Download the image
    print(f"Downloading square image for {file_name} at {center_lat}, {center_lon}...")
    response = requests.get(api_base_url, params=params)

    if response.status_code == 200:
        # Save the image
        with open(output_file, "wb") as file:
            file.write(response.content)
        print(f"Saved: {output_file}")
    else:
        print(f"Failed to download for {file_name} - HTTP {response.status_code}")

# Generate random points and download tiles
attempts = 0
tiles_downloaded = 0

while attempts < max_attempts:
    try:
        # Generate a random point outside the exclusion zones
        random_lat, random_lon = generate_random_point()
        file_name = f"tile_{random_lat:.1f}_{random_lon:.1f}"

        # Check if the location is ocean before downloading
        if is_ocean(random_lat, random_lon):
            # Download the image tile for this point
            download_tile(random_lat, random_lon, file_name)

            tiles_downloaded += 1
            print(f"Downloaded {tiles_downloaded} tile(s).")

            # Stop if enough tiles are downloaded
            if tiles_downloaded >= 1000:
                break
        else:
            print(f"Skipping tile at {random_lat}, {random_lon} - Not over ocean.")

    except Exception as e:
        print(f"Error during attempt {attempts}: {e}")
        attempts += 1

print(f"Download complete. {tiles_downloaded} tiles downloaded.")


Loaded 33452 records from seamounts.csv.
Exclusion zones defined for 33452 points.
Skipping tile at 42.855763044840955, -7.6802343823800925 - Not over ocean.
Skipping tile at 37.51821198809908, 39.234124750490025 - Not over ocean.
Skipping tile at 41.27464509925781, 123.3411795399171 - Not over ocean.
Saved: no_seamounts_galore/tile_77.1_-165.1.png
Downloaded 1 tile(s).
Saved: no_seamounts_galore/tile_-53.0_-179.1.png
Downloaded 2 tile(s).
Skipping tile at -76.28476677005196, 88.38723283703462 - Not over ocean.
Skipping tile at -87.2262504584397, -66.45383688427026 - Not over ocean.
Saved: no_seamounts_galore/tile_-69.7_-47.7.png
Downloaded 3 tile(s).
Saved: no_seamounts_galore/tile_66.0_-57.6.png
Downloaded 4 tile(s).
Saved: no_seamounts_galore/tile_-2.2_-118.8.png
Downloaded 5 tile(s).
Skipping tile at -15.980535638578758, 46.649933605841454 - Not over ocean.
Skipping tile at -78.74774215443621, -86.2357170196813 - Not over ocean.
Skipping tile at 64.1845641432846, 57.70654828050641 